In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-18.4")

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-06-30")
    ),
    granularity = TimeGranularity.MONTHLY
)

user

UserTO(id=a3238ec2-dbaf-43c2-ae89-836aae9b61c6, username=Johann-18.4)

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=9abc5e56-51ef-48d1-b548-590d66631248, name=Expenses)

In [4]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, dist

In [5]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import ro.jf.funds.reporting.api.model.ReportViewTaskStatus
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task.status == ReportViewTaskStatus.IN_PROGRESS && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task.status == ReportViewTaskStatus.COMPLETED) {
        task.report ?: error("No report found on completed report task")
    } else if (task.status == ReportViewTaskStatus.FAILED) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=bc5cb9ca-2a1d-45cd-971d-a39690b61b5f, name=Expense report, fundId=9abc5e56-51ef-48d1-b548-590d66631248, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(enabled=false, inputBuckets=0, outputBuckets=0))))

In [6]:
import ro.jf.funds.reporting.api.model.*

val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=bc5cb9ca-2a1d-45cd-971d-a39690b61b5f, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-06-30), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), bucketType=REAL, net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04, allocated=2085.009, left=-220.03099999999895), ReportDataGroupItemTO(group=home, net=-500.0, allocated=2085.009, left=1585.0090000000002), ReportDataGroupItemTO(group=shopping_services, net=-385.57, allocated=347.5015, left=-38.068499999999965), ReportDataGroupItemTO(group=transport, net=-204.24, allocated=347.5015, left=143.26150000000004), ReportDataGroupItemTO(group=fun, net=-1031.16, allocated=1042.5045, left=11.344499999999972), ReportDataGroupItemTO(group=gifts, net=0.0, allocated=834.0036, left=834.0035999999998), ReportDataGroupItemTO(group=development, net=-42.5, allocated=208.500

In [7]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun plotGroupData(plotTitle: String, groupItemFilter: (ReportDataGroupItemTO) -> Boolean): Plot =
    dataFrameOf(
        "month" to report.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.net!!.negate() }.reduce { acc, value -> acc + value } },
        "allocated" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value } },
        "left" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value } },
    )
        .plot {
            x("month")
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.YELLOW
                }
            }
            layout {
                title = plotTitle
                size = 1200 to 600
            }
        }

fun plotGroup(title: String, group: String): Plot =
    plotGroupData(title) { it.group == group }

fun plotGroupsTotal(): Plot =
    plotGroupData("Total expenses") { true }



In [8]:
plotGroupsTotal()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oi9zBE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Total expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[2481.520000000001,5185.079999999997,11301.259999999995,11526.016022686403,10643.310981435998,11868.237464847623,8637.902755956753,-6550.839311906871,-872.2869408013701,4051.763487254636,6037.119861628132,12394.376169342517,18628.67079533699,24174.78283797799,22045.408532850404,24075.588180919804,17645.084581462477,6743.3528066059425,11417.57697971895,11649.2164983658,13165.887815888522,14335.735712279346,15192.733397698072,15955.701192370567],
"spent":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,5442.03,35747.94,17363.44,18069.43,20971.82,13766.207215933333,13977.342817261111,13055.936544366205,13765.810548130052,14348.25309380756,14728.300851624854],
"allocated":[6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,11641.82,10066.59,13341.92529312179,13787.099900881942,14115.457392622104,14440.158548885487,14668.17676129261,14908.66482473366]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("oi9zBE");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 423.96581182238117 L50.59197647333349 423.96581182238117 L95.40201277828237 423.96581182238117 L135.87559395694916 423.96581182238117 L180.68563026190168 423.96581182238117 L224.05018152475895 423.96581182238117 L268.86021782970784 423.96581182238117 L312.2247690925651 423.96581182238117 L357.03480539751763 423.96581182238117 L401.84484170247015 423.9658118223811

In [9]:
plotGroup("Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2qqCOX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[-220.03099999999895,-627.8770000000007,610.4919999999985,331.1561012062398,509.73594555671764,438.7755442223798,-1100.4121419462283,-933.4796510148036,238.80813312598673,912.7238048757862,864.1489359274386,1318.0806887876365,1674.957422307332,2037.5014427911592,7440.435901812355,10565.855322135185,11062.06914559833,10031.963917586048,3676.054410165519,4074.093289508164,4491.391034551745,4845.772943003892,5173.5270378479,5532.641879674606],
"spent":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.1,2165.51,2324.86,948.25,1951.84,3154.12,2354.804166666667,2241.903680555556,2220.913987268519,2273.667652874228,2275.370790613748,2215.580023164893],
"allocated":[2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2514.8522,2529.8965,7733.9823,4073.0592,2444.7822,2113.9839,3152.644862476217,3235.369092682568,3283.429958739449,3327.120642824853,3360.548613060257,3395.129330815279]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"5"
};
 var containerDiv = document.getElementById("2qqCOX");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 454.4654782381597 L50.59197647333349 454.4654782381597 L95.40201277828237 454.4654782381597 L135.87559395694916 454.4654782381597 L180.68563026190168 454.4654782381597 L224.05018152475895 454.4654782381597 L268.86021782970784 454.4654782381597 L312.2247690925651 454.4654782381597 L357.03480539751763 454.4654782381597 L401.84484170247015 454.4654782381597 L

In [10]:
plotGroup("Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7K9KNf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[1585.0090000000002,3077.3129999999974,5899.701999999997,7800.108479624487,10008.067979312284,12174.271824801981,14278.272626912014,-1461.6718557107292,1391.5594627746714,4914.616339225614,8487.549802722746,12130.350898181561,15669.423109686319,16886.64913948618,-691.0011527930521,-9460.893004058495,-17917.016435055844,-31195.584264044675,1086.021222277192,-13.33306144237628,107.3618380799865,0.3453693554293886,-409.177211467086,-1150.571129316239],
"spent":[500.0,609.16,1763.9,400.0,400.0,467.23,420.0,18881.0,423.24,456.78,400.0,400.0,400.0,2736.62,29698.21,16136.26,12866.69,17070.89,8324.140833333333,8982.81923611111,8157.97083912037,8802.5317423804,9498.011054245435,10256.17864209922],
"allocated":[2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2559.9456,3150.9792,3270.09392558768,3979.3396,4005.9888,4041.7602,3936.2904,3959.838,12105.3636,7370.2976,4423.8916,3825.3042,4719.09106046564,4899.019848837777,5044.689902907591,5192.572901017584,5293.675676102383,5400.982915777581]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("7K9KNf");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.22922167989964 254.6766569976774 L50.22922167989964 254.6766569976774 L94.71796088209885 254.6766569976774 L134.90133822602365 254.6766569976774 L179.39007742822287 254.6766569976774 L222.44369601099606 254.6766569976774 L266.93243521319164 254.6766569976774 L309.98605379596484 254.6766569976774 L354.47479299816405 254.6766569976774 L398.96353220036326 254.6766569976774 L442.017

In [11]:
plotGroup("Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="mrx0vK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[143.26150000000004,285.65549999999996,-164.08299999999818,57.84894869290987,74.97220915129769,304.8544645207909,737.1299514409177,150.64249361617706,742.7057967534837,1792.634828863892,583.3937240473571,-129.4225174931565,574.0356044651923,1731.670041576009,5530.010285929626,8245.928901442958,8659.44094234711,9875.359150696464,3207.794100307169,3413.682779379357,3685.602803192955,3930.844220396244,4109.028336357274,4402.831220716433],
"spent":[204.24,207.85,1214.12,281.75,554.64,348.18,288.04,1474.32,331.47,238.42,2502.33,2020.41,608.51,161.2,230.49,0.0,1218.9,202.81,773.075,813.4945833333333,758.4257986111111,794.0054484953704,840.3042358699846,701.80208885915],
"allocated":[347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,719.9847,886.2129,919.713916571535,1287.4334,1296.0552,1307.6283,1312.0968,1319.946,4035.1212,2715.3728,1629.8548,1409.3226,1569.895218047628,1640.721094551597,1703.596777430897,1768.920349169177,1809.044261599942,1851.793350066603]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("mrx0vK");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 489.5771465805662 L50.59197647333349 489.5771465805662 L95.40201277828237 489.5771465805662 L135.87559395694916 489.5771465805662 L180.68563026190168 489.5771465805662 L224.05018152475895 489.5771465805662 L268.86021782970784 489.5771465805662 L312.2247690925651 489.5771465805662 L357.03480539751763 489.5771465805662 L401.84484170247015 489.5771465805662 L445.2093929653238 489.5771465805662 L490.

In [12]:
plotGroup("Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Q4wMw3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[-38.068499999999965,312.1755000000001,466.7869999999998,-582.457598613593,-1028.7034137877947,-3026.531795626687,-2838.1822606888645,-2047.7662370607488,-1318.578303415337,-728.3891642756969,217.09924487171645,1167.9317116627415,1674.2364562821936,2333.1803815478033,3085.123471868731,4055.2938726629986,4639.072347952559,5078.4451050369225,1276.455552203752,1619.34203661147,1924.934392750821,2195.227117431334,2438.861807573587,2624.008687798743],
"spent":[385.57,0.0,609.77,1481.1,936.99,2494.99,463.18,0.0,82.99,342.57,0.0,0.0,149.99,0.0,1259.78,0.0,0.0,69.0,197.2925,175.1352083333333,189.7298090277778,198.6247931134259,186.6293592062114,202.181805806729],
"allocated":[347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,639.9864,787.7448,817.52348139692,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,582.091,503.3295,871.9946984497433,891.3287233205554,899.9607169306016,906.8304865584084,904.3734271049424,901.1890793636876]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"14"
};
 var containerDiv = document.getElementById("Q4wMw3");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 320.4545668221798 L50.59197647333349 320.4545668221798 L95.40201277828237 320.4545668221798 L135.87559395694916 320.4545668221798 L180.68563026190168 320.4545668221798 L224.05018152475895 320.4545668221798 L268.86021782970784 320.4545668221798 L312.2247690925651 320.4545668221798 L357.03480539751763 320.4545668221798 L401.84484170247015 320.4545668221798 L445.2093929653238 320.4545668221798 L490.0194292702763 320.

In [13]:
plotGroup("Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ny1NXm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[11.344499999999972,492.0765000000009,1547.3309999999974,373.9688179422317,-800.390838407375,273.56292146943974,365.7608801590669,-531.9210572523725,-1167.5207547156358,-522.7118537654993,539.262175975756,1509.0409379958353,1695.7361750793923,2464.5853669028975,3524.251114625476,4494.500418015816,5078.490509557446,5587.412200467236,1919.740509420451,2049.2388118589,2264.335467618172,2550.32348614599,2806.409764471947,2995.338730179963],
"spent":[1031.16,570.0,1237.89,2109.0,2235.76,0.0,790.09,1980.318092,1753.8784992,405.5,0.0,100.0,579.01,0.0,1287.38,0.0,0.0,0.0,574.6813826,556.7306644833333,438.0983788569444,328.4500354950231,322.0292051196084,348.8649722129091],
"allocated":[1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,765.3898,769.9685,2353.8207,969.776,582.091,503.3295,1017.93681557673,1029.433108541458,1024.956775919913,1016.695275003175,1013.64033125344,1009.742958857893]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"17"
};
 var containerDiv = document.getElementById("ny1NXm");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 415.45500295284455 L50.59197647333349 415.45500295284455 L95.40201277828237 415.45500295284455 L135.87559395694916 415.45500295284455 L180.68563026190168 415.45500295284455 L224.05018152475895 415.45500295284455 L268.86021782970784 415.45500295284455 L312.2247690925651 415.45500295284455 L357.03480539751763 415.45500295284455 L401.84484170247015 415.45500295284455 L445.2093929653238 415

In [14]:
plotGroup("Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3JUNYU"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[834.0035999999998,1282.5891999999997,2969.2548,3429.72519422603,2478.5667171189198,2137.6155746152303,-2531.8216382972787,-1650.995880158403,-881.640980684099,265.67625340538643,-2439.3411812738595,-1742.6168368400504,-1306.9640195520465,-427.5569662723534,3129.3628167547363,5064.776013859649,4341.192452864764,4977.912421710903,566.4985379597789,824.6918859812004,1030.999199826167,1190.385881535356,1267.445017212854,1576.343867086747],
"spent":[0.0,392.0,147.85,402.0,1930.0,1330.0,5561.66,204.8,351.0,20.0,3892.49,491.19,766.57,330.2,141.92,198.28,2005.0,475.0,1203.175833333333,839.9688194444444,892.8995543981481,938.0578505979938,1014.56267148116,774.7353941045899],
"allocated":[834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2133.5072,1280.6002,1107.3249,1438.133623910064,1484.646317569236,1518.104419033339,1550.93855504272,1582.65060129628,1616.352151404303]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"20"
};
 var containerDiv = document.getElementById("3JUNYU");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 349.15415949375904 L50.59197647333349 349.15415949375904 L95.40201277828237 349.15415949375904 L135.87559395694916 349.15415949375904 L180.68563026190168 349.15415949375904 L224.05018152475895 349.15415949375904 L268.86021782970784 349.15415949375904 L312.2247690925651 349.15415949375904 L357.03480539751763 349.15415949375904 L401.84484170247015 349.15415949375904 L445.2093929653238 349.15415949375904 L490.01

In [15]:
plotGroup("Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DqNrc3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[166.0009,363.1472999999999,-28.223800000000587,115.66607960809735,-598.9376175080516,-434.311069155512,-272.8446616228733,-75.64712432599102,122.37970535955994,-2582.7867210748464,-2214.9928406430226,-1858.9887129520505,-1352.753952931392,-851.2465680537064,27.22609465253139,1110.1266568616916,1781.8356181981108,2387.8442751530442,-314.987352614912,-318.4992435309153,-338.7369201313256,-377.1633055888994,-193.3613542984038,-24.89206376968559],
"spent":[42.5,13.0,850.0,0.0,878.0,1.0,0.0,0.0,7.0,3060.0,0.0,0.0,40.0,48.5,805.3,80.65,27.0,0.0,339.0375,367.290625,397.8981770833333,430.4730251736111,211.345777271412,228.957925377363],
"allocated":[208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,1163.7312,698.5092,603.9954,572.2290141957692,606.5817153787499,640.7188416603124,677.0803392695693,704.2438508753667,733.475038448314]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"23"
};
 var containerDiv = document.getElementById("DqNrc3");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 280.5278313962256 L50.59197647333349 280.5278313962256 L95.40201277828237 280.5278313962256 L135.87559395694916 280.5278313962256 L180.68563026190168 280.5278313962256 L224.05018152475895 280.5278313962256 L268.86021782970784 280.5278313962256 L312.2247690925651 280.5278313962256 L357.03480539751763 280.5278313962256 L401.84484170247015 280.5278313962256 L445.2093929653238 280.5278313962256 L490.0194292702763 280.5278313962256 L533.

In [16]:
plotGroup("Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="0tzKOQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"26"
};
 var containerDiv = document.getElementById("0tzKOQ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M51.317486060193914 260.5 L51.317486060193914 260.5 L96.7701165706494 260.5 L137.8241054188038 260.5 L183.27673592926294 260.5 L227.26315255228474 260.5 L272.71578306274023 260.5 L316.70219968576566 260.5 L362.15483019622116 260.5 L407.60746070667665 260.5 L451.5938773297021 260.5 L497.0465078401576 260.5 L541.0329244631794 260.5 L586.4855549736385 260.5 L631.938185484094 260.5 L674.4583882196821 260.5 L719.9110187301376 260.5 L763.897435353163 260.5 L809.3500658636185 260.5 L853.3364824866403 260.5 L898.7891129970994 260.5 L944.2417435075549 260.5 L988.2281601305767 260.5 L1033.6807906410359 260.5 L1077.6672072640577 260.5 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M51.317486060193914 260.5 L51.317486060193914 260.5 L96.7701165706494 260.5 L137.8241054188038 260.5 L183.27673592926294 260.5 L227.26315255228474 260.5 L272.71578306274023 260.5 L316.70219968576566 260.5 L362.15483019622116 260.5 L407.60746070667665 260.5 L451.5938773297021 260.5 L497.0465078401576 260.5 L541.0329244631794 260.5 L586